In [ ]:
%load_ext autoreload

%autoreload 2

In [ ]:
from environment import *

import setting

path_dict = path(setting)

In [ ]:
feature_x_sample = pd.read_table(
    path_dict['feature_x_sample_file_path'],
    index_col=0,
)

In [ ]:
for element in (
    'feature',
    'sample',
):

    if element == 'feature':

        df = feature_x_sample

    elif element == 'sample':

        df = feature_x_sample.T

    skew_t_pdf_fit_parameter = ccal.fit_skew_t_pdfs(
        df,
        n_job=setting.MAX_N_JOB,
        directory_path=path_dict['{}_directory_path'.format(element)],
    )

    skew_t_pdf_fit_parameter = pd.read_table(
        path_dict['{}_skew_t_pdf_fit_parameter_file_path'.format(element)],
        index_col=0,
    )

    skew_t_pdf_fit_parameter['log(Degree of Freedom)'] = ccal.log_nd_array(skew_t_pdf_fit_parameter['Degree of Freedom'])

    skew_t_pdf_fit_parameter_ = skew_t_pdf_fit_parameter.drop(
        [
            'N',
            'Degree of Freedom',
        ],
        axis=1,
    )

    if skew_t_pdf_fit_parameter_.shape[0] < 1e3:

        for x_name_x, y_name_y in itertools.combinations(
            skew_t_pdf_fit_parameter_.items(),
            2,
        ):
            
            x_name, x = x_name_x
            
            y_name, y = y_name_y

            x = x.sort_values()

            y = y[x.index]

            title = '({}) {} vs {}'.format(
                element,
                x_name,
                y_name,
            )

            ccal.plot_points(
                (title, ),
                (x, ),
                (y, ),
                texts=(x.index, ),
                title=title,
                xaxis_title=x_name,
                yaxis_title=y_name,
            )

    for column, series in skew_t_pdf_fit_parameter_.items():

        ccal.plot_distributions(
            (column, ),
            (series, ),
            plot_rug=False,
            title='({}) {}'.format(
                element,
                column,
            ),
        )

        series_sorted = series.sort_values()

        index_min = series_sorted.index[0]

        index_max = series_sorted.index[-1]

        ccal.plot_distributions(
            (
                index_min,
                index_max,
            ),
            (
                df.loc[index_min].dropna(),
                df.loc[index_max].dropna(),
            ),
            plot_rug=False,
            title='({}) Extreme {}'.format(
                element,
                column,
            ),
            xaxis_title='Value',
        )

        n_extreme = 3

        for i in list(range(n_extreme)) + list(range(
            -n_extreme,
            0,
        )):

            index = series_sorted.index[i]

            series = df.loc[index]

            location, scale, degree_of_freedom, shape = skew_t_pdf_fit_parameter.loc[
                index, [
                    'Location',
                    'Scale',
                    'Degree of Freedom',
                    'Shape',
                ]]

            ccal.plot_context(
                series,
                location=location,
                scale=scale,
                degree_of_freedom=degree_of_freedom,
                shape=shape,
                plot_rug=series.dropna().size < 1e3,
                title='({}) {} Rank {} [{}]'.format(
                    element,
                    column,
                    i,
                    index,
                ),
            )